In [1]:
import pandas as pd
import gomill
import gomill.sgf
from thirdparty.GoBoard import GoBoard
from visualization.BoardEvaluator import BoardEvaluator
from board_evaluation.model_eval import print_info

In [2]:
# get vaild sgf files
f = pd.read_csv('./sqlresult_3240554.csv')

valid_sgf = []
count = 0
for i in f['sgf']:
    if '又' in i:
        count += 1
        valid_sgf.append(i)

print(count)

195


In [3]:
board_evaluator = BoardEvaluator('./data/working/cnn_5layer_64filter', 9)

INFO:tensorflow:Restoring parameters from ./data/working/cnn_5layer_64filter


In [4]:
def get_final_state(sgf_str):
    sgf = gomill.sgf.Sgf_game.from_string(sgf_str)
    board = GoBoard(9)
    sgf_iterator = sgf.main_sequence_iter()
    
    color_to_move = 'b'
    while True:
        try:
            it = sgf_iterator.next()
            color, move = it.get_move()
            if color is None:
                it = sgf_iterator.next()
                color, move = it.get_move()
        except StopIteration:
            break

        color_to_move = color
        if move is not None:
            (row, col) = move
            board.applyMove(color, (row, col))

    color_to_move = 'b' if color_to_move == 'w' else 'w'
#     print(str(board))
#     print('color to move: %s' % color_to_move)
    
    return board, color_to_move

In [5]:
def compare_result(board_evaluator, sgf_str):
    board, color_to_move = get_final_state(sgf_str)
    y_val = board_evaluator.evaluate_board(board, color_to_move)
    print("y_pre:      y_val:      y_true:     error:")
    error_count = print_info(None, (y_val > 0.5).astype(int), y_val, board.get_final_ownership('b'), 9)
    return error_count

In [8]:
bad_result = []
for f in valid_sgf:
    e = compare_result(board_evaluator, f)
    if e > 4:
        bad_result.append(f)

y_pre:      y_val:      y_true:     error:
111111111   999999999   111111000   ......XXX
111111111   999999999   111110000   .....XXXX
111111111   999999999   110011000   ..XX..XXX
111111111   999999999   111111000   ......XXX
111111111   999999999   111111000   ......XXX
111111111   999999999   011110010   X....XX.X
111111111   999999999   011100111   X...XX...
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
y_pre:      y_val:      y_true:     error:
111111111   999999999   111111111   .........
111111111   999999999   111101111   ....X....
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
y_pre:      y_val:      y_true:     error:
111111000   999999000   111111000   .......

000000111   000000999   000000111   .........
000000011   000000099   000000011   .........
000000011   000000099   000000011   .........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
y_pre:      y_val:      y_true:     error:
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000101   ......X.X
000000000   000000000   000000101   ......X.X
000000000   000000000   000000000   .........
y_pre:      y_val:      y_true:     error:
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111100   999999900   111111100   ....

y_pre:      y_val:      y_true:     error:
111111111   899999999   011111111   X........
111111111   798999999   000111111   XXX......
111111111   899999999   000011111   XXXX.....
111111111   999999999   000001111   XXXXX....
111110111   999992999   000000111   XXXXX....
111111111   999999599   000010011   XXXX.XX..
111111111   999999999   000111111   XXX......
111111111   999999999   000111111   XXX......
100011111   533399999   000011111   X........
y_pre:      y_val:      y_true:     error:
000011111   000099999   000011111   .........
000001111   000009999   000001111   .........
000011111   000099999   000011111   .........
000001111   000009999   000001111   .........
000011111   000099999   000011111   .........
000000111   000000999   000000111   .........
000000111   000000999   000000111   .........
000001111   000009999   000001111   .........
000000111   000000999   000000111   .........
y_pre:      y_val:      y_true:     error:
000000000   000000000   000010000   ....X..

y_pre:      y_val:      y_true:     error:
000000000   000000000   000001100   .....XX..
000000000   000000000   000000100   ......X..
000000000   000000000   000000000   .........
000000000   000000000   000000100   ......X..
000000000   000000000   000000000   .........
000000000   000000000   000000001   ........X
000000000   000000000   000000000   .........
000000000   000000000   100001100   X....XX..
000000000   000000000   110000000   XX.......
y_pre:      y_val:      y_true:     error:
111111111   999999999   111101110   ....X...X
111111111   999999999   111111110   ........X
111111111   999999999   111111111   .........
111111111   999999999   111111101   .......X.
000011111   000099999   000011111   .........
000001111   000129999   000001111   .........
000011111   000099999   000011111   .........
000111111   000689999   000001110   ...XX...X
000011111   000099999   000011111   .........
y_pre:      y_val:      y_true:     error:
000000000   000000000   000000000   .......

y_pre:      y_val:      y_true:     error:
001111111   009999999   001111111   .........
000111110   000999990   000111110   .........
000011100   000099900   000011100   .........
000011100   000099900   000011100   .........
000011000   000099000   000011000   .........
000111000   000999000   000111000   .........
000111000   000999000   000111000   .........
000111100   000999900   000111100   .........
000011000   000099000   000011000   .........
y_pre:      y_val:      y_true:     error:
000000000   000000000   110000000   XX.......
000000000   000000000   110000000   XX.......
000000000   000000000   100000000   X........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000111   000000999   000000111   .........
000010111   000090999   000010111   .........
011111111   099999999   011111111   .........
y_pre:      y_val:      y_true:     error:
000000000   000000000   000000000   .......

y_pre:      y_val:      y_true:     error:
000000000   000000000   110011000   XX..XX...
000000000   000000000   000000000   .........
000000000   000000000   000000001   ........X
000000000   000000000   000000000   .........
000000000   000000000   100011100   X...XXX..
000000000   000000000   110000000   XX.......
000000000   000000000   110010000   XX..X....
000000000   000000000   100000010   X......X.
000000000   000000000   100000110   X.....XX.
y_pre:      y_val:      y_true:     error:
111111111   999999999   111111111   .........
111111111   999999999   111101111   ....X....
111111111   999999999   111110111   .....X...
111111111   999999999   101111111   .X.......
111111111   999999999   101110011   .X...XX..
111111111   999999999   111110011   .....XX..
111111111   999999999   111111111   .........
111111111   999999999   111110111   .....X...
111111111   999999999   111110111   .....X...
y_pre:      y_val:      y_true:     error:
000011111   000099999   000011111   .......

In [9]:
for f in bad_result:
    e = compare_result(board_evaluator, f)

y_pre:      y_val:      y_true:     error:
111111111   999999999   111111000   ......XXX
111111111   999999999   111110000   .....XXXX
111111111   999999999   110011000   ..XX..XXX
111111111   999999999   111111000   ......XXX
111111111   999999999   111111000   ......XXX
111111111   999999999   011110010   X....XX.X
111111111   999999999   011100111   X...XX...
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
y_pre:      y_val:      y_true:     error:
000000000   000000000   011111110   .XXXXXXX.
000000000   000000000   000110110   ...XX.XX.
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   100000000   X........
001000000   009000000   001000000   .........
011110000   099990000   011110000   .........
111111000   999999000   111111000   .........
111110000   999990000   111110000   .........
y_pre:      y_val:      y_true:     error:
111111111   999999999   111111100   .......

y_pre:      y_val:      y_true:     error:
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000010   .......X.
000000000   000000000   000001111   .....XXXX
000000000   000000000   000000111   ......XXX
000000000   000000000   000000010   .......X.
000000000   000000000   000001110   .....XXX.
000000000   000000000   000000100   ......X..
000000000   000000000   000000000   .........
y_pre:      y_val:      y_true:     error:
000000000   000000000   000110001   ...XX...X
000000000   000000000   110111001   XX.XXX..X
000000000   000000000   110110001   XX.XX...X
000000000   000000000   110110000   XX.XX....
000000000   000000000   010010000   .X..X....
000000000   000000000   000111000   ...XXX...
000000000   000000000   000001100   .....XX..
000000000   000000000   000001000   .....X...
000000000   000000000   000011000   ....XX...
y_pre:      y_val:      y_true:     error:
111111111   999999999   111110000   .....XX